files bias$k ("bias"+str(k)) are computed by reprocessing the demuxed concateneted trajectory for each replica.

In [1]:
import plumed
import numpy as np

Bias=[]
for k in range(8):
    bias=plumed.read_as_pandas("bias"+str(k))
    MEW=plumed.read_as_pandas("MEbias"+str(k))
    bias["sum"]=bias["metad5.bias"]
    for i in ((6,7,8,9,10,11,12,13,14,15,16,25,26,27,28,29,30,31,32,33,34,35,36)):
        bias["sum"]+=bias["metad"+str(i)+".bias"]
    bias["sum"]+=(MEW["JCI9"]+MEW["JCI10"]+MEW["JCU11"]+MEW["JCI12"]+MEW["JCU23"]+MEW["JCU29"]+MEW["JCI30"]+MEW["JCU31"]+MEW["JCU32"])
    Bias.append(bias["sum"])
Bias=np.array(Bias)
Bias=np.transpose(Bias)
for j in range(8):
    Bias[:,j]-=np.min(Bias[:,j])
Bias 

/u/v/vpiompon/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: cannot load PLUMED instance, conversions will not be available
  


FileNotFoundError: [Errno 2] No such file or directory: 'bias0'

In [8]:
from bussilab import wham
import plumed
import numpy as np
Bias=np.load('Bias.npy')

a =wham.wham(bias=Bias, T=2.494, maxiter=10000)

Back-calculate J-couplings from the trajecotry using Barnaba python package. Davies parameters for the J_H1H2 must be implemented directly in the barnaba code (Condon parameters are the default values).

The demuxed concatenated trajecotry was divided in 5 parts here in orfer to allow loading 

In [10]:
from importlib.machinery import SourceFileLoader
bb = SourceFileLoader("barnbaba","/net/sbp/sbpstore1/vpiompon/barnaba/barnaba/__init__.py").load_module()
bb

<module 'barnbaba' from '/net/sbp/sbpstore1/vpiompon/barnaba/barnaba/__init__.py'>

In [11]:
top = "rna_ions.pdb"
list=[]
 
list_couplings=[]
for k in (0,1,2,3,4,5,6,7):
# define trajectory and topology files
    traj = "traj_"+str(k)+"_a.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_b.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_c.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_d.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_e.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings)
        
        
    
    print(couplings.shape)

OSError: No such file: traj_0_a.xtc

In [3]:
list_couplings=np.concatenate((list_couplings[0],list_couplings[1]))
Exp=[0.84,4.624,5.52,2.71,0.834,1.568,5.752,5.056,5.136,1.95**2]

IndexError: list index out of range

In [12]:
list_couplings=np.load('Davies_couplings.npy')
Exp=[0.84,4.624,5.52,2.71,0.834,1.568,5.752,5.056,5.136,1.95**2]

MD+NMR ensemble

In [18]:
import sys
print(sys.path)

['/net/sbp/sbpstore1/vpiompon/Inosine/I_dsRNA/ME_bias+RECT_MD', '/net/sbp/sbpstore1/vpiompon/Inosine/I_dsRNA/ME_bias+RECT_MD', '/net/sbp/sbpstore1/vpiompon/py-bussilab/bussilab', '/u/v/vpiompon/miniconda3/lib/python37.zip', '/u/v/vpiompon/miniconda3/lib/python3.7', '/u/v/vpiompon/miniconda3/lib/python3.7/lib-dynload', '', '/u/v/vpiompon/.local/lib/python3.7/site-packages', '/u/v/vpiompon/.local/lib/python3.7/site-packages/ParmEd-3.4.0+34.gc11babc-py3.7-linux-x86_64.egg', '/u/v/vpiompon/miniconda3/lib/python3.7/site-packages', '/u/v/vpiompon/miniconda3/lib/python3.7/site-packages/pmx-2.0+38.g32e0eac-py3.7-linux-x86_64.egg', '/u/v/vpiompon/miniconda3/lib/python3.7/site-packages/IPython/extensions', '/u/v/vpiompon/.ipython', '/net/sbp/sbpstore1/vpiompon/py-bussilab/bussilab/']


In [16]:
sys.path.append("/net/sbp/sbpstore1/vpiompon/py-bussilab/bussilab/")

In [17]:
from bussilab import maxent

b=maxent.maxent(traj=np.array(list_couplings).reshape(-1,9), reference=Exp[0:9], logW=a.logW)

OSError: libcublas.so.10: cannot open shared object file: No such file or directory

In [6]:
nw=np.exp(-np.sum(((list_couplings).reshape(-1,9)*b.lambdas),axis=1))
np.save("Pure_MD_ensemble/wei.npy",np.exp(a.logW))
np.save("MD+NMR_ensemble/wei.npy",np.exp(a.logW)*nw)

AttributeError: 'list' object has no attribute 'reshape'

MD+NMR+SAXS ensemble

In [7]:
rG=plumed.read_as_pandas("colvar_rg")
np.shape(np.array(rG['rg']))

/u/v/vpiompon/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: cannot load PLUMED instance, conversions will not be available
  """Entry point for launching an IPython kernel.


(2800008,)

In [8]:
traj_values=np.concatenate((np.array(list_couplings).reshape(-1,9),np.square(np.array(rG['rg']))[:,np.newaxis]),axis=1)

In [9]:
from bussilab import maxent

b=maxent.maxent(traj=traj_values, reference=Exp, logW=a.logW)
nw=np.exp(-np.sum(((traj_values).reshape(-1,10)*b.lambdas),axis=1))


In [7]:
np.save("MD+NMR+SAXS_ensemble/wei.npy",np.exp(a.logW)*nw)

NameError: name 'a' is not defined